In [1]:
import os
os.getcwd()
os.chdir("H:\pyspark_advanced-coding_interview")
print(os.getcwd())


from pyspark.sql import SparkSession

# Create a Spark session with optimized settings
spark = (
    SparkSession.builder 
    .appName("OptimizedLocalSpark") 
    .config("spark.driver.memory", "8g")        
    .config("spark.executor.memory", "8g")    
    .config("spark.executor.cores", "4")       
    .config("spark.cores.max", "12")           
    .config("spark.sql.shuffle.partitions", "28")  
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") 
    .getOrCreate()
)
sc = spark.sparkContext

H:\pyspark_advanced-coding_interview


# create a Christmas Tree

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, expr

# Initialize SparkSession
spark = SparkSession.builder.appName("ChristmasTree").getOrCreate()

# Create a base DataFrame for the tree trunk
df = spark.createDataFrame([(1, "*")], ["level", "pattern"])

# Generate the tree pattern recursively
for i in range(2, 11):  # Adjust range for desired tree height
    new_row = df.withColumn("level", lit(i)).withColumn("pattern", expr("repeat('*', level)"))
    df = df.union(new_row)

# Show the tree
df.orderBy("level").show(truncate=False)


In [ ]:
# Create a base DataFrame and register as a temporary table
base_df = spark.sql("SELECT 1 AS level, '*' AS pattern")
base_df.createOrReplaceTempView("tree_base")

# Use Spark SQL to create the recursive CTE
tree_sql = """
WITH RECURSIVE tree_cte(level, pattern) AS (
    SELECT level, pattern FROM tree_base
    UNION ALL
    SELECT level + 1, repeat('*', level + 1)
    FROM tree_cte
    WHERE level < 10  -- Adjust the value for tree height
)
SELECT * FROM tree_cte ORDER BY level
"""

# Run the query and show the result
tree_df = spark.sql(tree_sql)
tree_df.show(truncate=False)


In [ ]:
from pyspark.sql import Row

# Generate rows for the Christmas tree
tree_rows = [Row(level=i, pattern='*' * i) for i in range(1, 11)]  # Adjust range for tree height

# Create DataFrame from rows
tree_df = spark.createDataFrame(tree_rows)

# Show the DataFrame
tree_df.orderBy("level").show(truncate=False)
